In [1]:
%pip install pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, DoubleType
from pyspark.sql.functions import count, avg, format_number, desc, asc,col, split, explode, max
sp = SparkSession.builder.master("local").appName("my-spark-proj").config(conf=SparkConf()).getOrCreate()

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


23/12/05 00:58:21 WARN Utils: Your hostname, MacBook-Pro-Pasa.local resolves to a loopback address: 127.0.0.1; using 192.168.1.35 instead (on interface en0)
23/12/05 00:58:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/05 00:58:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
def show_funct():
    name_basics = sp.read.csv('./data/name.basics.tsv.gz', header=True, sep='\t')
    title_akas = sp.read.csv('./data/title.akas.tsv.gz', header=True, sep='\t')
    title_basics = sp.read.csv('./data/title.basics.tsv.gz', header=True, sep='\t')
    title_crew = sp.read.csv('./data/title.crew.tsv.gz', header=True, sep='\t')
    title_episode = sp.read.csv('./data/title.episode.tsv.gz', header=True, sep='\t')
    title_principals = sp.read.csv('./data/title.principals.tsv.gz', header=True, sep='\t')
    title_ratings = sp.read.csv('./data/title.ratings.tsv.gz', header=True, sep='\t')

    print("name_basics data:")
    name_basics.show(5)
    print("akas data:")
    title_akas.show(5)
    print("title_basics data:")
    title_basics.show(5)
    print("crew data:")
    title_crew.show(5)
    print("episodes data:")
    title_episode.show(5)
    print("principals data:")
    title_principals.show(5)
    print("ratings data:")
    title_ratings.show(5)
show_funct()

name_basics data:
+---------+---------------+---------+---------+--------------------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0053137,tt00319...|
|nm0000002|  Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0037382,tt00383...|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0049189,tt00573...|
|nm0000004|   John Belushi|     1949|     1982|actor,soundtrack,...|tt0080455,tt00725...|
|nm0000005| Ingmar Bergman|     1918|     2007|writer,director,a...|tt0069467,tt00839...|
+---------+---------------+---------+---------+--------------------+--------------------+
only showing top 5 rows

akas data:
+---------+--------+--------------------+------+--------+-----------+-------------+---------------+
|  titleId|ordering|               t

In [3]:
def create_scheme_and_display_dfs(name_basics_path, akas_path, basics_path, crew_path, episode_path, principals_path,
                                  ratings_path):
    name_basics_scheme = StructType([
        StructField("nconst", StringType(), True),
        StructField("primaryName", StringType(), True),
        StructField("birthYear", StringType(), True),
        StructField("deathYear", StringType(), True),
        StructField("primaryProfession", StringType(), True),
        StructField("knownForTitles", StringType(), True)
    ])
    akas_scheme = StructType([
        StructField("titleId", StringType(), True),
        StructField("ordering", IntegerType(), True),
        StructField("title", StringType(), True),
        StructField("region", StringType(), True),
        StructField("language", StringType(), True),
        StructField("types", StringType(), True),
        StructField("attributes", StringType(), True),
        StructField("isOriginalTitle", BooleanType(), True),
    ])
    title_basics_scheme = StructType([
        StructField("tconst", StringType(), True),
        StructField("titleType", StringType(), True),
        StructField("primaryTitle", StringType(), True),
        StructField("originalTitle", StringType(), True),
        StructField("isAdult", IntegerType(), True),
        StructField("startYear", StringType(), True),
        StructField("endYear", StringType(), True),
        StructField("runtimeMinutes", IntegerType(), True),
        StructField("genres", StringType(), True),
    ])
    crew_scheme = StructType([
        StructField("tconst", StringType(), True),
        StructField("directors", StringType(), True),
        StructField("writers", StringType(), True)
    ])

    episode_scheme = StructType([
        StructField("tconst", StringType(), True),
        StructField("parentTconst", StringType(), True),
        StructField("seasonNumber", IntegerType(), True),
        StructField("episodeNumber", IntegerType(), True)
    ])

    principals_scheme = StructType([
        StructField("tconst", StringType(), True),
        StructField("ordering", IntegerType(), True),
        StructField("nconst", StringType(), True),
        StructField("category", StringType(), True),
        StructField("job", StringType(), True),
        StructField("characters", StringType(), True),
    ])

    ratings_scheme = StructType([
        StructField("tconst", StringType(), True),
        StructField("averageRating", DoubleType(), True),
        StructField("numVotes", IntegerType(), True)
    ])

    # Read compressed files directly into DataFrames
    name_basics_df = sp.read.option("delimiter", "\t").csv(name_basics_path, header=True, schema=name_basics_scheme)
    title_akas_df = sp.read.option("delimiter", "\t").csv(akas_path, header=True, schema=akas_scheme)
    title_basics_df = sp.read.option("delimiter", "\t").csv(basics_path, header=True, schema=title_basics_scheme)
    title_crew_df = sp.read.option("delimiter", "\t").csv(crew_path, header=True, schema=crew_scheme)
    title_episode_df = sp.read.option("delimiter", "\t").csv(episode_path, header=True, schema=episode_scheme)
    title_principals_df = sp.read.option("delimiter", "\t").csv(principals_path, header=True, schema=principals_scheme)
    title_ratings_df = sp.read.option("delimiter", "\t").csv(ratings_path, header=True, schema=ratings_scheme)

    # Show the DataFrames
    name_basics_df.show()
    title_basics_df.show()
    title_akas_df.show()
    title_crew_df.show()
    title_episode_df.show()
    title_principals_df.show()
    title_ratings_df.show()

    return name_basics_df, title_akas_df, title_basics_df, title_crew_df, title_episode_df, title_principals_df, title_ratings_df

name_basics_df, title_akas_df, title_basics_df, title_crew_df, title_episode_df, title_principals_df, title_ratings_df = create_scheme_and_display_dfs(
    "./data/name.basics.tsv.gz", "./data/title.akas.tsv.gz", "./data/title.basics.tsv.gz",
    "./data/title.crew.tsv.gz", "./data/title.episode.tsv.gz", "./data/title.principals.tsv.gz",
    "./data/title.ratings.tsv.gz"
)


+---------+-------------------+---------+---------+--------------------+--------------------+
|   nconst|        primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+-------------------+---------+---------+--------------------+--------------------+
|nm0000001|       Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0053137,tt00319...|
|nm0000002|      Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0037382,tt00383...|
|nm0000003|    Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0049189,tt00573...|
|nm0000004|       John Belushi|     1949|     1982|actor,soundtrack,...|tt0080455,tt00725...|
|nm0000005|     Ingmar Bergman|     1918|     2007|writer,director,a...|tt0069467,tt00839...|
|nm0000006|     Ingrid Bergman|     1915|     1982|actress,soundtrac...|tt0038787,tt00381...|
|nm0000007|    Humphrey Bogart|     1899|     1957|actor,soundtrack,...|tt0037382,tt00432...|
|nm0000008|      Marlon Brando|     1924|     2004|actor,sou

In [ ]:
# Name Basics show
num_columns = len(name_basics_df.columns)
num_rows = name_basics_df.count()

name_basics_df.printSchema()

print("\nColumns - ", name_basics_df.columns)

print("\n Count of columns -", num_columns)

print("\nNumber of rows - ", num_rows)

print("\nDescribe:")
name_basics_df.describe().show()
name_basics_df.explain(mode='extended')

root
 |-- nconst: string (nullable = true)
 |-- primaryName: string (nullable = true)
 |-- birthYear: string (nullable = true)
 |-- deathYear: string (nullable = true)
 |-- primaryProfession: string (nullable = true)
 |-- knownForTitles: string (nullable = true)


Columns -  ['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession', 'knownForTitles']

 Count of columns - 6

Number of rows -  13048362

Describe:


+-------+---------+--------------------+-----------------+------------------+--------------------+-------------------+
|summary|   nconst|         primaryName|        birthYear|         deathYear|   primaryProfession|     knownForTitles|
+-------+---------+--------------------+-----------------+------------------+--------------------+-------------------+
|  count| 13048362|            13048362|         13048362|          13048362|            10448505|           13048362|
|   mean|     NULL|                 NaN| 1952.96337448147|1992.6899380846685|                NULL|               NULL|
| stddev|     NULL|                 NaN|34.90875797958213| 34.53603376328906|                NULL|               NULL|
|    min|nm0000001|!'aru Ikhuisi Pie...|             0001|              0017|               actor|                 \N|
|    max|nm9993719|  ﻿Thesia Kouloungou|               \N|                \N|writer,visual_eff...|tt9916856,tt7297222|
+-------+---------+--------------------+--------

In [4]:
# Title Ratings show
num_columns = len(title_ratings_df.columns)
num_rows = title_ratings_df.count()

title_ratings_df.printSchema()

print("\nColumns - ", title_ratings_df.columns)

print("\n Count of columns -", num_columns)

print("\nNumber of rows - ", num_rows)

print("\nDescribe:")
title_ratings_df.describe().show()
title_ratings_df.explain(mode='extended')

root
 |-- tconst: string (nullable = true)
 |-- averageRating: double (nullable = true)
 |-- numVotes: integer (nullable = true)


Columns -  ['tconst', 'averageRating', 'numVotes']

 Count of columns - 3

Number of rows -  1374013

Describe:


+-------+---------+------------------+------------------+
|summary|   tconst|     averageRating|          numVotes|
+-------+---------+------------------+------------------+
|  count|  1374013|           1374013|           1374013|
|   mean|     NULL| 6.958483435018095| 1037.978271675741|
| stddev|     NULL|1.3832747322493522|17593.254139984583|
|    min|tt0000001|               1.0|                 5|
|    max|tt9916880|              10.0|           2823319|
+-------+---------+------------------+------------------+

== Parsed Logical Plan ==
Relation [tconst#478,averageRating#479,numVotes#480] csv

== Analyzed Logical Plan ==
tconst: string, averageRating: double, numVotes: int
Relation [tconst#478,averageRating#479,numVotes#480] csv

== Optimized Logical Plan ==
Relation [tconst#478,averageRating#479,numVotes#480] csv

== Physical Plan ==
FileScan csv [tconst#478,averageRating#479,numVotes#480] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/U

In [5]:
# Principals show
num_columns = len(title_principals_df.columns)
num_rows = title_principals_df.count()

title_principals_df.printSchema()

print("\nColumns - ", title_principals_df.columns)

print("\n Count of columns -", num_columns)

print("\nNumber of rows - ", num_rows)

print("\nDescribe:")
title_principals_df.describe().show()
title_principals_df.explain(mode='extended')

23/12/05 00:59:51 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


root
 |-- tconst: string (nullable = true)
 |-- ordering: integer (nullable = true)
 |-- nconst: string (nullable = true)
 |-- category: string (nullable = true)
 |-- job: string (nullable = true)
 |-- characters: string (nullable = true)


Columns -  ['tconst', 'ordering', 'nconst', 'category', 'job', 'characters']

 Count of columns - 6

Number of rows -  59313435

Describe:


+-------+---------+-----------------+---------+--------+------------------+-------------------+
|summary|   tconst|         ordering|   nconst|category|               job|         characters|
+-------+---------+-----------------+---------+--------+------------------+-------------------+
|  count| 59313435|         59313435| 59313435|59313435|          59313435|           59313435|
|   mean|     NULL|4.613686848519227|     NULL|    NULL|              NULL|               NULL|
| stddev|     NULL|2.787261670917683|     NULL|    NULL|              NULL|               NULL|
|    min|tt0000001|                1|nm0000001|   actor|"A Box in Town" by|["!CF","CF","Self"]|
|    max|tt9916880|               10|nm9993718|  writer|         écrit par|                 \N|
+-------+---------+-----------------+---------+--------+------------------+-------------------+

== Parsed Logical Plan ==
Relation [tconst#466,ordering#467,nconst#468,category#469,job#470,characters#471] csv

== Analyzed Logical Pl

In [6]:
# Episode show
num_columns = len(title_episode_df.columns)
num_rows = title_episode_df.count()

title_episode_df.printSchema()

print("\nColumns - ", title_episode_df.columns)

print("\n Count of columns -", num_columns)

print("\nNumber of rows - ", num_rows)

print("\nDescribe:")
title_episode_df.describe().show()
title_episode_df.explain(mode='extended')

root
 |-- tconst: string (nullable = true)
 |-- parentTconst: string (nullable = true)
 |-- seasonNumber: integer (nullable = true)
 |-- episodeNumber: integer (nullable = true)


Columns -  ['tconst', 'parentTconst', 'seasonNumber', 'episodeNumber']

 Count of columns - 4

Number of rows -  7901188

Describe:


+-------+---------+------------+------------------+-----------------+
|summary|   tconst|parentTconst|      seasonNumber|    episodeNumber|
+-------+---------+------------+------------------+-----------------+
|  count|  7901188|     7901188|           6295324|          6295324|
|   mean|     NULL|        NULL| 3.738206008141916|403.0755111254004|
| stddev|     NULL|        NULL|24.042708175598683|  1301.8276790122|
|    min|tt0041951|   tt0035599|                 1|                0|
|    max|tt9916880|   tt9916678|              2021|            91334|
+-------+---------+------------+------------------+-----------------+

== Parsed Logical Plan ==
Relation [tconst#458,parentTconst#459,seasonNumber#460,episodeNumber#461] csv

== Analyzed Logical Plan ==
tconst: string, parentTconst: string, seasonNumber: int, episodeNumber: int
Relation [tconst#458,parentTconst#459,seasonNumber#460,episodeNumber#461] csv

== Optimized Logical Plan ==
Relation [tconst#458,parentTconst#459,seasonNumber#4

In [7]:
# Crew show
num_columns = len(title_crew_df.columns)
num_rows = title_crew_df.count()

title_crew_df.printSchema()

print("\nColumns - ", title_crew_df.columns)

print("\n Count of columns -", num_columns)

print("\nNumber of rows - ", num_rows)

print("\nDescribe:")
title_crew_df.describe().show()
title_crew_df.explain(mode='extended')

root
 |-- tconst: string (nullable = true)
 |-- directors: string (nullable = true)
 |-- writers: string (nullable = true)


Columns -  ['tconst', 'directors', 'writers']

 Count of columns - 3

Number of rows -  10351957

Describe:


+-------+---------+-------------------+-------------------+
|summary|   tconst|          directors|            writers|
+-------+---------+-------------------+-------------------+
|  count| 10351957|           10351957|           10351957|
|   mean|     NULL|               NULL|               NULL|
| stddev|     NULL|               NULL|               NULL|
|    min|tt0000001|                 \N|                 \N|
|    max|tt9916880|nm9993709,nm9993708|nm9993713,nm3125335|
+-------+---------+-------------------+-------------------+

== Parsed Logical Plan ==
Relation [tconst#452,directors#453,writers#454] csv

== Analyzed Logical Plan ==
tconst: string, directors: string, writers: string
Relation [tconst#452,directors#453,writers#454] csv

== Optimized Logical Plan ==
Relation [tconst#452,directors#453,writers#454] csv

== Physical Plan ==
FileScan csv [tconst#452,directors#453,writers#454] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users

In [8]:
# Title Basics show
num_columns = len(title_basics_df.columns)
num_rows = title_basics_df.count()

title_basics_df.printSchema()

print("\nColumns - ", title_basics_df.columns)

print("\n Count of columns -", num_columns)

print("\nNumber of rows - ", num_rows)

print("\nDescribe:")
title_basics_df.describe().show()
title_basics_df.explain(mode='extended')

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: integer (nullable = true)
 |-- startYear: string (nullable = true)
 |-- endYear: string (nullable = true)
 |-- runtimeMinutes: integer (nullable = true)
 |-- genres: string (nullable = true)


Columns -  ['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres']

 Count of columns - 9

Number of rows -  10351957

Describe:


+-------+---------+---------+--------------+-------------+--------------------+------------------+------------------+------------------+--------+
|summary|   tconst|titleType|  primaryTitle|originalTitle|             isAdult|         startYear|           endYear|    runtimeMinutes|  genres|
+-------+---------+---------+--------------+-------------+--------------------+------------------+------------------+------------------+--------+
|  count| 10351957| 10351957|      10351957|     10351957|            10351956|          10351957|          10351957|           3113900|10351939|
|   mean|     NULL|     NULL|           NaN|          NaN|0.034995415359184295|2005.4540555160538| 2006.454020793918|43.417214746780566|    NULL|
| stddev|     NULL|     NULL|           NaN|          NaN|  2.5903709298061344| 20.12947392889537|16.394336427194403| 74.11959410113813|    NULL|
|    min|tt0000001|    movie|        !Next?|       !Next?|                   0|              1874|              1906|       

In [9]:
# Title Akas show
num_columns = len(title_akas_df.columns)
num_rows = title_akas_df.count()

title_akas_df.printSchema()

print("\nColumns - ", title_akas_df.columns)

print("\n Count of columns -", num_columns)

print("\nNumber of rows - ", num_rows)

print("\nDescribe:")
title_akas_df.describe().show()
title_akas_df.explain(mode='extended')

root
 |-- titleId: string (nullable = true)
 |-- ordering: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- region: string (nullable = true)
 |-- language: string (nullable = true)
 |-- types: string (nullable = true)
 |-- attributes: string (nullable = true)
 |-- isOriginalTitle: boolean (nullable = true)


Columns -  ['titleId', 'ordering', 'title', 'region', 'language', 'types', 'attributes', 'isOriginalTitle']

 Count of columns - 8

Number of rows -  37983582

Describe:


+-------+---------+-----------------+--------+--------+--------+--------+------------------+
|summary|  titleId|         ordering|   title|  region|language|   types|        attributes|
+-------+---------+-----------------+--------+--------+--------+--------+------------------+
|  count| 37983582|         37983582|37983582|37983579|37983579|37983579|          37983579|
|   mean|     NULL|4.168659027471396|     NaN|    NULL|    NULL|    NULL|              NULL|
| stddev|     NULL|3.996599797010896|     NaN|    NULL|    NULL|    NULL|              NULL|
|    min|tt0000001|                1|   !Huff|      AD|      \N|      \N|16mm release title|
|    max|tt9916856|              250|🧠+🧘=❤️|      \N|      zu| working|     weekend title|
+-------+---------+-----------------+--------+--------+--------+--------+------------------+

== Parsed Logical Plan ==
Relation [titleId#418,ordering#419,title#420,region#421,language#422,types#423,attributes#424,isOriginalTitle#425] csv

== Analyzed Logica

In [ ]:
#Business questions -----
#1 - Скільки фільмів в кожному жанрі мають найбільшу кількість ролей?
#2 - Фільму у якому було задіяно найбільше ролей
#3 - Який макс. рейтинг по кожному жанру
#4 - Які топ 5 фільмів мають найвищий середній рейтинг, але при цьому отримали більше 500 голосів?
#5 - Які режисери мають найвищий середній рейтинг та зняли більше 10 фільмів?
#6 - Які фільми отримали найменше голосів та який в них рейтинг?
#7 - Яка популярність телесеріалів за жанрами та регіонами, використовуючи середній рейтинг та кількість телесеріалів у кожній категорії.


In [15]:
#1 - Яка загальна кількість фільмів для кожного жанру?
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, count, max as spark_max

spark = SparkSession.builder.appName("RoleCount").getOrCreate()
title_basics_df = spark.read.option("header", "true").csv("./data/title.basics.tsv.gz", sep="\t")
title_principals_df = spark.read.option("header", "true").csv("./data/title.principals.tsv.gz", sep="\t")

title_basics_df = title_basics_df.withColumn("genres", split(col("genres"), ","))

movies_per_genre = title_basics_df.select("tconst", explode("genres").alias("genre")) \
                                   .groupBy("genre") \
                                   .agg(count("tconst").alias("movie_count")) \
                                   .orderBy("movie_count", ascending=False)

movies_per_genre.show(truncate=False)

+-----------+-----------+
|genre      |movie_count|
+-----------+-----------+
|Drama      |2941672    |
|Comedy     |2062912    |
|Talk-Show  |1268874    |
|Short      |1131255    |
|Romance    |979220     |
|Documentary|976771     |
|News       |908714     |
|Family     |765485     |
|Reality-TV |580085     |
|Animation  |520423     |
|\N         |460965     |
|Crime      |422487     |
|Action     |422288     |
|Adventure  |395927     |
|Music      |394116     |
|Game-Show  |371459     |
|Adult      |323732     |
|Sport      |246519     |
|Fantasy    |208465     |
|Mystery    |208288     |
+-----------+-----------+
only showing top 20 rows



In [33]:
#2 - Фільму у якому було задіяно найбільше ролей
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, count, max as spark_max


spark = SparkSession.builder.appName("MostRoles").getOrCreate()


title_principals_df = spark.read.option("header", "true").csv("./data/title.principals.tsv.gz", sep="\t")
title_basics_df = spark.read.option("header", "true").csv("./data/title.basics.tsv.gz", sep="\t")
title_principals_df = title_principals_df.withColumn("characters", split(col("characters"), ",")) \
                                         .withColumn("character", explode(col("characters")))

merged_df = title_principals_df.join(title_basics_df, on="tconst")
roles_per_film = merged_df.groupBy("tconst", "genres", "primaryTitle").agg(count("tconst").alias("role_count"))\
    .groupBy("tconst", "genres", "primaryTitle").agg(spark_max("role_count").alias("max_role_count")).orderBy("max_role_count", ascending=False)

roles_per_film.show(truncate=False)

23/12/05 02:09:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/05 02:09:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/05 02:09:23 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/05 02:09:23 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/05 02:09:25 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/05 02:09:25 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/05 02:09:29 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/05 02:09:29 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/05 02:09:31 WARN RowBasedKeyValueBatch: Calling spill() on

+----------+------------------------+---------------------------+--------------+
|tconst    |genres                  |primaryTitle               |max_role_count|
+----------+------------------------+---------------------------+--------------+
|tt7066416 |Comedy                  |High Land Parking          |63            |
|tt0423652 |Crime,Mystery,Reality-TV|The First 48               |57            |
|tt14923012|News,Talk-Show          |The Media Show             |50            |
|tt21036670|Reality-TV              |Coastguard Dutch Caribbean |50            |
|tt0441059 |Adventure,Comedy,Fantasy|Kaamelott                  |48            |
|tt6225166 |Crime,Reality-TV        |Live PD                    |47            |
|tt0139803 |Music                   |Top of the Pops            |46            |
|tt15452764|News,Talk-Show          |Valg 2021                  |46            |
|tt6500140 |News,Talk-Show          |Heads Up                   |46            |
|tt10098498|Documentary     

In [26]:
#3 - Який макс. рейтинг по кожному жанру

spark = SparkSession.builder.appName("max_ratings_per_genre").getOrCreate()

title_df = spark.read.option("header", "true").csv("./data/title.basics.tsv.gz", sep="\t")
ratings_df = spark.read.option("header", "true").csv("./data/title.ratings.tsv.gz", sep="\t")

joined_df = title_df.join(ratings_df, ["tconst"])
joined_df = joined_df.withColumn("genres_array", split(col("genres"), ","))
genres_df = joined_df.select("tconst", "averageRating", explode("genres_array").alias("genre"))
max_ratings_per_genre = genres_df.groupBy("genre").agg(max(col("averageRating")).alias("maxRating"))

max_ratings_per_genre.show()

23/12/05 01:57:26 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-----------+---------+
|      genre|maxRating|
+-----------+---------+
|     Action|      9.9|
|      Adult|      9.9|
|  Adventure|      9.9|
|  Animation|      9.9|
|  Biography|      9.9|
|     Comedy|      9.9|
|      Crime|      9.9|
|Documentary|      9.9|
|      Drama|      9.9|
|     Family|      9.9|
|    Fantasy|      9.9|
|  Film-Noir|      8.4|
|  Game-Show|      9.9|
|    History|      9.9|
|     Horror|      9.9|
|      Music|      9.9|
|    Musical|      9.9|
|    Mystery|      9.9|
|       News|      9.9|
| Reality-TV|      9.9|
+-----------+---------+
only showing top 20 rows



In [29]:
#4 - Які топ 5 фільмів мають найвищий середній рейтинг, але при цьому отримали більше 500 голосів?

spark = SparkSession.builder.appName("Top5Films").getOrCreate()

filtered_ratings = title_ratings_df.filter(col("numVotes") > 500)
joined_df = filtered_ratings.join(title_basics_df, "tconst")
top_5_films_rating = joined_df.orderBy(col("averageRating").desc()).limit(5)

top_5_films_rating.select("primaryTitle", "averageRating", "numVotes").show()

+-------------------+-------------+--------+
|       primaryTitle|averageRating|numVotes|
+-------------------+-------------+--------+
|         Ozymandias|         10.0|  208858|
|           Prologue|         10.0|     615|
|              Beega|         10.0|     843|
|      Episode #1.40|         10.0|     522|
|End of The Prologue|          9.9|   15109|
+-------------------+-------------+--------+



In [35]:
#5 - Які режисери мають найвищий середній рейтинг та зняли більше 10 фільмів?
spark = SparkSession.builder.appName("DirectorsRating").getOrCreate()
ratings_df = title_ratings_df.join(title_crew_df, "tconst")
movies_per_director = ratings_df.groupBy("directors").agg(count("tconst").alias("filmCount"))
directors_with_more_than_10_films = movies_per_director.filter(col("filmCount") > 10)
movies_with_ratings = ratings_df.join(directors_with_more_than_10_films, "directors")
director_stats = movies_with_ratings.groupBy("directors").agg(
    avg("averageRating").alias("avgRating"),
    count("tconst").alias("filmCount")
)
director_stats_df = director_stats.withColumn("avgRating", format_number("avgRating", 2))
sorted_director_stats_df = director_stats_df.orderBy(col("avgRating").desc())
sorted_director_stats_df.show()

+--------------------+---------+---------+
|           directors|avgRating|filmCount|
+--------------------+---------+---------+
|nm15185869,nm1667...|     9.99|       35|
|           nm9697887|     9.99|       25|
|           nm1475126|     9.97|      497|
|nm5870567,nm58887...|     9.96|       27|
|          nm11066250|     9.94|       17|
|           nm8331592|     9.92|      155|
|           nm2448669|     9.91|       16|
|           nm3466713|     9.90|       26|
|           nm2320575|     9.88|       13|
|           nm1425932|     9.84|       27|
|           nm1341249|     9.83|       21|
|           nm3148014|     9.83|       46|
|           nm3795227|     9.83|       13|
| nm9343671,nm2375975|     9.83|       47|
|          nm13078903|     9.78|       13|
|          nm13389794|     9.78|       11|
|          nm11256194|     9.76|       48|
|           nm3116146|     9.76|       36|
|           nm4779082|     9.76|      106|
|          nm11521823|     9.75|       42|
+----------

In [38]:
#6 - Які фільми отримали найменше голосів та який в них рейтинг?
spark = SparkSession.builder.appName("LessVotedFilms").getOrCreate()

title_ratings = spark.read.option("delimiter", "\t").csv("./data/title.ratings.tsv.gz", header=True, inferSchema=True)
title_basics = spark.read.option("delimiter", "\t").csv("./data/title.basics.tsv.gz", header=True, inferSchema=True)

merged_data = title_ratings.join(title_basics, title_ratings["tconst"] == title_basics["tconst"])
less_voted_movies = merged_data.sort(asc("numVotes")).select("originalTitle", "numVotes", "averageRating")

less_voted_movies.show(truncate=False)

23/12/05 02:19:08 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-----------------------------------+--------+-------------+
|originalTitle                      |numVotes|averageRating|
+-----------------------------------+--------+-------------+
|Neutralidad                        |5       |6.0          |
|Figuras de la Pasión               |5       |6.8          |
|Váltás                             |5       |7.8          |
|Zonnetje                           |5       |4.6          |
|Pälsen                             |5       |5.6          |
|La defensa de Madrid               |5       |6.8          |
|Hoje, Estreia                      |5       |6.2          |
|División heroica                   |5       |6.0          |
|It All Goes to Show                |5       |5.8          |
|Ein drittes Reich aus seinem Abfall|5       |6.0          |
|Leyenda                            |5       |4.2          |
|Río Negro                          |5       |6.2          |
|Kahden tanssin välillä             |5       |3.8          |
|Bonecos de Estremoz    

In [54]:
#7 - Яка популярність телесеріалів за жанрами та регіонами, використовуючи середній рейтинг та кількість телесеріалів у кожній категорії.
spark = SparkSession.builder.appName("TVGenresPopularity").getOrCreate()

akas = spark.read.option("delimiter", "\t").csv("./data/title.akas.tsv.gz", header=True, inferSchema=True)
ratings = spark.read.option("delimiter", "\t").csv("./data/title.ratings.tsv.gz", header=True, inferSchema=True)
title_basics = spark.read.option("delimiter", "\t").csv("./data/title.basics.tsv.gz", header=True, inferSchema=True)

merged_data = title_basics.join(akas, title_basics["tconst"] == akas["titleId"]).join(ratings, "tconst")

tv_series_data = merged_data.filter(col("titleType") == "tvSeries")

genre_region_popularity = tv_series_data.groupBy("genres", "region").agg(
    avg("averageRating").alias("avgRating"),
    count("*").alias("numSeries")
)
genre_region_popularity_formated = genre_region_popularity.withColumn("avgRating",format_number("avgRating", 2))
genre_region_popularity_formated.show()



23/12/05 02:43:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/05 02:43:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/05 02:43:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/05 02:43:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/05 02:43:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/05 02:43:07 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/05 02:43:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/05 02:43:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/05 02:43:10 WARN RowBasedKeyValueBatch: Calling spill() on

+--------------------+------+---------+---------+
|              genres|region|avgRating|numSeries|
+--------------------+------+---------+---------+
|              Family|    JP|     6.89|       55|
|     Game-Show,Music|    \N|     5.58|      120|
|         Crime,Drama|    IT|     7.10|      213|
|   Adventure,History|    BG|     7.75|        2|
|             Western|  SUHH|     7.69|       23|
|        Comedy,Music|    FI|     6.21|       15|
|       Drama,Romance|    FI|     6.69|       54|
|Action,Adventure,...|    IT|     7.01|       71|
|Adventure,Comedy,...|    IN|     6.78|       20|
|             Mystery|    FR|     6.86|       15|
|Drama,Mystery,Sci-Fi|    PL|     7.13|       38|
|Drama,Family,Thri...|    ES|     7.70|        1|
|               Drama|    TR|     6.05|      602|
|    Animation,Family|    BR|     6.95|      129|
|Drama,Fantasy,Sci-Fi|    PT|     6.90|        2|
|Action,Adventure,...|    HR|     6.77|       13|
|              Comedy|    TW|     7.28|      158|


In [55]:
movies_per_genre.write.csv("./data/loaded_data/movie_count", header= True)
roles_per_film.limit(20).write.csv("./data/loaded_data/roles_per_film", header= True)
max_ratings_per_genre.limit(20).write.csv("./data/loaded_data/max_ratings_per_genre", header= True)
top_5_films_rating.write.csv("./data/loaded_data/top_5_films_rating", header= True)
sorted_director_stats_df.limit(20).write.csv("./data/loaded_data/sorted_director_stats_df", header= True)
less_voted_movies.limit(20).write.csv("./data/loaded_data/less_voted_movies", header= True)
genre_region_popularity_formated.write.csv("./data/loaded_data/genre_region_popularity", header= True)

23/12/05 02:45:23 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/05 02:45:23 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/05 02:45:24 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/05 02:45:26 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/05 02:45:26 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/05 02:45:27 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/05 02:45:28 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/05 02:45:29 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/12/05 02:45:30 WARN RowBasedKeyValueBatch: Calling spill() on